In [1]:
%matplotlib inline
import numpy as np
import scipy.linalg as la
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree
from poly_basis import *

# Parameters

In [2]:
n = 200
center_index = 30

rbf_labels = ['r^3', 'r^4*log(r)', 'r^5', 'r^6*log(r)', 'r^7', 'r^8*log(r)', 'r^9', 'gaussian']
epsilon = 1

#rbf_label = 'r^4*log(r)'
rbf_label = rbf_labels[2]

harm_index = 20

deg_diff = 0

print('n: %d' % n)
print('forcing deg: %d' % sphere_harm_degs[harm_index])

n: 200
forcing deg: 4


In [3]:
basis_deg = sphere_harm_degs[harm_index] + deg_diff
stencil_size = 2*(basis_deg+1)**2 #101
assert stencil_size > (basis_deg+1)**2
assert n >= stencil_size
print('Stencil Size: %d' % stencil_size)

Stencil Size: 50


# Solutions

In [4]:
harm_deg = sphere_harm_degs[harm_index]

foo = lambda x: -harm_deg*(harm_deg+1)*sphere_harm[harm_index](*x)
exact = lambda x: sphere_harm[harm_index](*x)
exact_x = lambda x: sphere_harm_grad_x[harm_index](*x)
exact_y = lambda x: sphere_harm_grad_y[harm_index](*x)
exact_z = lambda x: sphere_harm_grad_z[harm_index](*x)

# RBFs

In [5]:
rbf_dict = {}

even_tol = 1e-14

def rbf(r):
    return r**3
def d_phi_dr_div_r(r):
    return 3 * r
rbf_dict['r^3'] = (rbf, d_phi_dr_div_r)

def rbf(r):
    if abs(r)< even_tol:
        return 0
    return r**4 * np.log(r)
def d_phi_dr_div_r(r):
    if abs(r)< even_tol:
        return 0
    return r**2 * (1 + 4*np.log(r))
rbf_dict['r^4*log(r)'] = (rbf, d_phi_dr_div_r)

def rbf(r):
    return r**5
def d_phi_dr_div_r(r):
    return 5 * r**3
rbf_dict['r^5'] = (rbf, d_phi_dr_div_r)

def rbf(r):
    if abs(r)< even_tol:
        return 0
    return r**6 * np.log(r)
def d_phi_dr_div_r(r):
    if abs(r)< even_tol:
        return 0
    return r**4 * (1 + 6*np.log(r))
rbf_dict['r^6*log(r)'] = (rbf, d_phi_dr_div_r)

def rbf(r):
    return r**7
def d_phi_dr_div_r(r):
    return 7 * r**5
rbf_dict['r^7'] = (rbf, d_phi_dr_div_r)

def rbf(r):
    if abs(r)< even_tol:
        return 0
    return r**8 * np.log(r)
def d_phi_dr_div_r(r):
    if abs(r)< even_tol:
        return 0
    return r**6 * (1 + 8*np.log(r))
rbf_dict['r^8*log(r)'] = (rbf, d_phi_dr_div_r)

def rbf(r):
    return r**9
def d_phi_dr_div_r(r):
    return 9 * r**7
rbf_dict['r^9'] = (rbf, d_phi_dr_div_r)

# RBFs with Shape Parameter
shape_rbfs = []

def rbf(r):
    return np.exp(-(epsilon*r)**2)
def d_phi_dr_div_r(r):
    return -2 * epsilon**2 * np.exp(-(epsilon*r)**2)
rbf_dict['gaussian'] = (rbf, d_phi_dr_div_r)
shape_rbfs += ['gaussian']



def grad_rbf(node, node_center):
    r = dist(node, node_center)
    return d_phi_dr_div_r(r) * (np.array(node) - np.array(node_center))

########################################################################################
#
# misc
#
########################################################################################

def dist(node1, node2):
    return np.sqrt( (node1[0]-node2[0])**2 + (node1[1]-node2[1])**2 + (node1[2]-node2[2])**2 )

# Calculate Gradient Weights

In [6]:
# generate nodes
indices = np.arange(0, n, dtype=float) + 0.5
phi = np.arccos(1 - 2*indices/n)
theta = np.pi * (1 + 5**0.5) * indices
xs, ys, zs = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi)
nodes = np.array([(x,y,z) for x,y,z in zip(xs,ys,zs)])
projectors = [np.eye(3) - np.outer(node, node) for node in nodes]

# generate stencils
tree = cKDTree(np.array(nodes))


# generate weights
rbf, d_phi_dr_div_r = rbf_dict[rbf_label]
stencils = [tree.query(node, stencil_size)[1] for node in nodes]
weights_LB = np.zeros(stencil_size)

stencil = stencils[center_index]

nn = np.array([nodes[i] for i in stencil])
nn_proj = [projectors[i] for i in stencil]
center_x, center_y, center_z = nn[0]
P, rhs_x, rhs_y, rhs_z = gen_sphere_harm_basis(basis_deg, nn, nn_proj)
A = np.array([[rbf(dist(node, node_center)) for node in nn] for node_center in nn])
num_basis = P.shape[1]
AP = np.block([[A,P],[P.T, np.zeros((num_basis,num_basis))]])
rhsAs = np.array([ [ nn_proj[j]@grad_rbf(node, node_center) for j, node in enumerate(nn) ]
                                    for node_center in nn] )

rhsA = rhsAs[:,:,0] # only the x coordinates
rhs = np.block([[rhsA],[rhs_x]])
weights_grad_x = la.solve(AP, rhs)[:stencil_size,:].T
weights_LB = (weights_grad_x@weights_grad_x)[0]

rhsA = rhsAs[:,:,1] # only the y coordinates
rhs = np.block([[rhsA],
                [rhs_y]])
weights_grad_y = la.solve(AP, rhs)[:stencil_size,:].T
weights_LB += (weights_grad_y@weights_grad_y)[0]

rhsA = rhsAs[:,:,2] # only the z coordinates
rhs = np.block([[rhsA],
                [rhs_z]])
weights_grad_z = la.solve(AP, rhs)[:stencil_size,:].T
weights_LB += (weights_grad_z@weights_grad_z)[0]

# Difference in Surface Gradient

In [7]:
surf_grad = np.array([ weights_grad_x[0] @ [exact(x) for x in nn],
           weights_grad_y[0] @ [exact(x) for x in nn],
           weights_grad_z[0] @ [exact(x) for x in nn]])

exact_grad = np.array([exact_x(nodes[center_index]), 
                       exact_y(nodes[center_index]), 
                       exact_z(nodes[center_index])])
exact_surf_grad = projectors[center_index]@ exact_grad

print(surf_grad - exact_surf_grad)

[-1.88737914e-14  2.66453526e-15 -1.15463195e-14]


# Difference in Laplace-Beltrami

In [8]:
weights_LB @ [exact(x) for x in nn] - foo(nodes[center_index])

0.01849153186057606